In [ ]:
from helpers.util_functions import central_difference
from lodegp.LODEGP import LODEGP
import torch
import gpytorch


In [ ]:
train_x = torch.linspace(0r, 1r, 1r)
train_y = torch.linspace(0r, 1r, 1r)
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=3r)
model = LODEGP(train_x, train_y, ODE_name="Heating", likelihood=likelihood, num_tasks=3r)


In [ ]:
model.sage_locals

In [ ]:
def extract_differential_polynomial_terms(expr, diff_var, var_dict):
    """
    Parses a polynomial expression in the differential operator variable (e.g., x),
    and returns a dict mapping derivative order to coefficient.

    Parameters:
    - expr: Sage symbolic expression (e.g., x^2 + 981/100)
    - diff_var: the Sage variable representing the differential operator (e.g., x)

    Returns:
    - dict: {order: coefficient}  (e.g., {0: 981/100, 2: 1})
    """
    expr = sage_eval(str(expr), locals=var_dict)
    if type(expr) is not sage.symbolic.expression.Expression:
        return {0: expr}
    terms = expr.coefficients(diff_var)
    result = {}

    for term in terms:
        # each term is a tuple [coefficient, degree]
        coeff = term[0]
        degree = term[1]
        result[degree] = coeff

    return result


In [ ]:
# Verify that the functions satisfy the given differential equation
def calculate_differential_equation_error_symbolic(functions, differential_eq, sage_locals, **kwargs):
    # We know we that the channel count is equal to the number of tasks
    dx = kwargs.get("diff_var", var("x"))
    differential_equation_error = 0
    for i, column in enumerate(differential_eq):
        # Each channel contains the polynom of differentials that is used on the respective channel
        # Dictionary of the form {order: coeff}
        coeff_dict = extract_differential_polynomial_terms(column, dx, sage_locals)
        for order, coeff in coeff_dict.items():
            differential_equation_error += coeff*functions[i].diff(dx, int(order))
    return differential_equation_error

# Verify that the given data satisfies the given differential equation
def calculate_differential_equation_error_numeric(differential_eq, sage_locals, data_generating_functions, **kwargs):
    dx = kwargs.get("diff_var", var("x"))
    # We know we that the channel count is equal to the number of tasks
    channel_values = [[] for _ in range(len(differential_eq))]
    for column in differential_eq:
        # Each channel contains the polynom of differentials that is used on the respective channel
        # Dictionary of the form {order: coeff}
        coeff_dict = extract_differential_polynomial_terms(column, dx, sage_locals)


        pass
    pass



# Verify that the models output satisfies the given differential equation
target_row = 0
target_col = 0
model_mean_generator = lambda x: model(x).mean
calculate_differential_equation_error_numeric(model.A[target_row], model.sage_locals, model_mean_generator)

# Verify that the symbolic covariance functions satisfy the given differential equation
model_diffed_kernel_col = [model.diffed_kernel[i][target_col] for i in range(len(model.diffed_kernel))]
diff_var = var("t1")
var("x")
differential_equation = [term.substitute(x=diff_var) for term in sage_eval(str(model.A[target_row]), locals=model.sage_locals)]
calculate_differential_equation_error_symbolic(model_diffed_kernel_col, differential_equation, model.sage_locals, diff_var=diff_var)(t1=1, t2=1, signal_variance_2=1.0, lengthscale_2=1.0, a=1.0, b=1.0)

In [ ]:
model = LODEGP(train_x, train_y, ODE_name="Bipendulum", likelihood=likelihood, num_tasks=3r)
var("x")
for row in model.A:
    for column in row:
        print(column)
        print(extract_differential_polynomial_terms(column, x, model.sage_locals))